In [1]:
!pip install scikit-learn==0.22.2 --user

     |████████████████████████████████| 7.1 MB 2.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.22.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.2 which is incompatible.


In [1]:
!pip install pycrf
!pip install sklearn-crfsuite

import spacy
import sklearn_crfsuite
from sklearn_crfsuite import metrics

model = spacy.load("en_core_web_sm")

  Created wheel for pycrf: filename=pycrf-0.0.1-py3-none-any.whl size=1897 sha256=600148a481b658fb88acc750a6ed2989fdb86cf4980eca223ea63f18cf8e1247
  Stored in directory: /root/.cache/pip/wheels/0b/68/37/a457e156cfd6174ed28c9c8cb76f18eeb559b760d84c0a22eb
Successfully built pycrf
     |████████████████████████████████| 965 kB 4.1 MB/s 


In [3]:
def combining_into_sentences(location):
    list_of_all_sentences=[]
    with open(location,"r") as file:
        temp_string=""
        for each_word in file:
            if each_word!="\n":
                temp_string=temp_string+" "+(each_word) 
            if each_word=="\n":
                list_of_all_sentences.append(temp_string)
                temp_string=""
    final_cleaned_sentences=[]
    for each_sentence in list_of_all_sentences:
        final_cleaned_sentences.append(each_sentence.replace("\n","").strip())
    return final_cleaned_sentences

In [4]:
location_train="/content/train_sent"
training_sentences=combining_into_sentences(location_train)

location_test="/content/test_sent"
test_sentences=combining_into_sentences(location_test)

location_train_label="/content/train_label"
training_labels=combining_into_sentences(location_train_label)
location_test_label="/content/test_label"
test_labels=combining_into_sentences(location_test_label)

In [5]:
### Printing top 5 sentences 
for i in range(1,6):
    print("Sentence",i,"",training_sentences[i-1])
    print(len(training_sentences[i-1]))
    print("Label",i,"",training_labels[i-1])
    print(len(training_labels[i-1]))

Sentence 1  All live births > or = 23 weeks at the University of Vermont in 1995 ( n = 2395 ) were retrospectively analyzed for delivery route , indication for cesarean , gestational age , parity , and practice group ( to reflect risk status )
231
Label 1  O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
89
Sentence 2  The total cesarean rate was 14.4 % ( 344 of 2395 ) , and the primary rate was 11.4 % ( 244 of 2144 )
100
Label 2  O O O O O O O O O O O O O O O O O O O O O O O O O
49
Sentence 3  Abnormal presentation was the most common indication ( 25.6 % , 88 of 344 )
75
Label 3  O O O O O O O O O O O O O O O
29
Sentence 4  The `` corrected '' cesarean rate ( maternal-fetal medicine and transported patients excluded ) was 12.4 % ( 273 of 2194 ) , and the `` corrected '' primary rate was 9.6 % ( 190 of 1975 )
187
Label 4  O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
75
Sentence 5  Arrest of dilation was the most co

In [6]:
nouns_tokens=[]
for each_sentence in training_sentences:
    nlp_sentence=model(each_sentence)
    for each_token in nlp_sentence:
        if each_token.pos_ =='NOUN' or each_token.pos_ =='PROPN':
            nouns_tokens.append(each_token)

for each_sentence in test_sentences:
    nlp_sentence=model(each_sentence)
    for each_token in nlp_sentence:
        if each_token.pos_ =='NOUN' or each_token.pos_ =='PROPN':
            nouns_tokens.append(each_token)

In [7]:
# Defining a functions to get all the features of all the words in a sentence
def featuresofallwords(sentence,pos):
    transformed_nlp_sentence=model(sentence)
    transformed_sentence_pos=[]
    features_list_per_sentence=[]
    for each_word in transformed_nlp_sentence:
        transformed_sentence_pos.append(each_word.pos_)
    sentence_to_list=sentence.split(" ")
    word = sentence_to_list[pos]
    word_lower=word.lower()
    last_three_characters=word[-3:]
    last_two_characters=word[-2:]
    word_upper=word.upper()
    word_is_digit=str(word.isdigit())
    word_starts_withcapital=str(word[0].isupper())
    word_pos=transformed_sentence_pos[pos]
    
    features_list_per_sentence=[word_lower,last_three_characters,last_two_characters,
                                     word_upper,word_is_digit,word_starts_withcapital,word_pos]
    

    if(pos > 0):
            prev_word = sentence_to_list[pos-1]
            previous_word_lower=prev_word.lower()
            previous_word_isupper=str(prev_word.isupper())
            previous_word_startswithcapital=str(prev_word[0].isupper())
            
            features_list_per_sentence.extend([previous_word_lower,previous_word_isupper,previous_word_startswithcapital])
    else:
         features_list_per_sentence.append('BEG')
    if (pos==len(sentence_to_list)-1):
        features_list_per_sentence.append('END')
            
    return features_list_per_sentence
    

In [8]:
# function to extract features given a document of sentences it takes in individual words and gives the features
def individual_sentences_features_generation(sentence_count):
    list_of_features_of_all_words_in1_sentence=[]
    for i in range(len(training_sentences[sentence_count].split(" "))):
        list_of_features_of_all_words_in1_sentence.append(featuresofallwords(training_sentences[sentence_count],i)) 

In [9]:
# Write a code to get features for a sentence.
 
def getFeaturesForOneSentence(sentence_count):
    list_of_features_of_all_words_in1_sentence=[]
    for i in range(len(training_sentences[sentence_count].split(" "))):
        list_of_features_of_all_words_in1_sentence.append(featuresofallwords(training_sentences[sentence_count],i))
    return  list_of_features_of_all_words_in1_sentence  

In [10]:
#Getting all the features of all the senetnces of training set
list_of_all_sentences_features=[]
for sentence_count in range(len(training_sentences)):
    list_of_all_sentences_features.append(getFeaturesForOneSentence(sentence_count))

In [11]:
# function to get all the feature of all the sentences in test set
def getFeaturesForOneSentence_test(sentence_count):
    list_of_features_of_all_words_in1_sentence=[]
    for i in range(len(test_sentences[sentence_count].split(" "))):
        list_of_features_of_all_words_in1_sentence.append(featuresofallwords(test_sentences[sentence_count],i))
    return  list_of_features_of_all_words_in1_sentence  

In [12]:
# Getting all the features of the test set
list_of_all_sentences_features_test=[]
for sentence_count in range(len(test_sentences)):
    list_of_all_sentences_features_test.append(getFeaturesForOneSentence_test(sentence_count))

In [13]:
# Write a code to get the labels for a sentence.
def getLabelsInListForOneSentence(training_labels):
    list_of_labels=[]
    for i in range(len(training_labels)):
            list_of_labels.append(training_labels[i])
    return list_of_labels

In [14]:
X_train_own=list_of_all_sentences_features
X_test_own=list_of_all_sentences_features_test

In [15]:
y_train=getLabelsInListForOneSentence(training_labels)
y_test=getLabelsInListForOneSentence(test_labels)

In [16]:
y_train_new=[]
y_test_new=[]
for each_label in y_train:
    y_train_new.append(each_label.split(" "))
for each_label in y_test:
    y_test_new.append(each_label.split(" "))

In [18]:
X_train_own[0][0]

['all', 'All', 'll', 'ALL', 'False', 'True', 'DET', 'BEG']

In [19]:
y_train_new[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [17]:
crf = sklearn_crfsuite.CRF(max_iterations=100)
crf.fit(X_train_own,y_train_new)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm=None, all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [3]:
#!pip install scikit-learn==0.22.0

     |████████████████████████████████| 7.0 MB 4.1 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.22 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22 which is incompatible.


In [18]:
predicted_tags=crf.predict(X_test_own)

In [19]:
metrics.flat_f1_score(y_test_new,predicted_tags,average='weighted')

0.9013443782511653

In [90]:
disease_dictionary=dict()
disease_list=[]
for i in range(len(predicted_tags)):
  temp_list=[]
  temp_string=""
  for j in range(len(predicted_tags[i])):
    if predicted_tags[i][j]=='D':
      #print(predicted_tags[i])
      #print(test_sentences[i])
      #print(test_sentences[i].split(" ")[j])
      temp_list.append(test_sentences[i].split(" ")[j])
      temp_string=" ".join(temp_list)
  disease_list.append(temp_string)
    


In [36]:
treatment_list=[]
for i in range(len(predicted_tags)):
  temp_list=[]
  temp_string=""
  for j in range(len(predicted_tags[i])):
    if predicted_tags[i][j]=='T':
      #print(predicted_tags[i])
      #print(test_sentences[i])
      #print(test_sentences[i].split(" ")[j])
      temp_list.append(test_sentences[i].split(" ")[j])
      temp_string=" ".join(temp_list)
  treatment_list.append(temp_string)
    

In [108]:
new_disease_list=[]
new_treatment_list=[]
for i in range(len(disease_list)):
  if (len(disease_list[i])==0) or (len(treatment_list[i])==0):
    pass
  else:
    new_disease_list.append(disease_list[i])
    new_treatment_list.append(treatment_list[i])

In [109]:
new_disease_list[:5]

['nonimmune hydrops fetalis',
 'unstable angina or non-Q-wave myocardial infarction',
 'cardiac disease',
 "early Parkinson 's disease",
 'sore throat']

In [110]:
new_treatment_list[:5]

['Trisomy',
 'roxithromycin',
 'fenfluramine-phentermine',
 'Ropinirole monotherapy',
 'Antibiotics']

In [111]:
## Creating a DataFrame
import pandas as pd
diseases_treatment=pd.DataFrame()
diseases_treatment['diseases']=new_disease_list
diseases_treatment['treatment']=new_treatment_list

In [112]:
diseases_treatment.head()

,diseases,treatment
0,nonimmune hydrops fetalis,Trisomy
1,unstable angina or non-Q-wave myocardial infar...,roxithromycin
2,cardiac disease,fenfluramine-phentermine
3,early Parkinson 's disease,Ropinirole monotherapy
4,sore throat,Antibiotics


In [113]:
diseases_treatment[diseases_treatment['diseases']=='hereditary retinoblastoma']

,diseases,treatment
